In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import os
import re
import string
import pprint
import json
import time

In [2]:
data_dir = "C:\\data"
xls_path = os.path.join(data_dir,"company_list.xls")

In [3]:
df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]


In [4]:
company_object = df['회사명']
print(company_object)
company_list = list(company_object)

0            DSR
1          GS글로벌
2          HSD엔진
3          LG이노텍
4           LS산전
          ...   
2350       에이원알폼
2351       원바이오젠
2352     지앤이헬스케어
2353          코셋
2354    한국미라클피플사
Name: 회사명, Length: 2355, dtype: object


In [5]:
from collections import defaultdict
company_dict = defaultdict(list)

In [6]:
# html = urlopen("https://www.amazon.com/books-used-books-textbooks/b?ie=UTF8&node=283155")  
url ="https://biz.chosun.com"
html = urlopen(url)
bsObject = BeautifulSoup(html, "html.parser") 
bsObject_lxml = BeautifulSoup(html, 'lxml', from_encoding='utf-8')

# print(bsObject.prettify())

In [7]:
web_driver_path = "C:\\project\\code\\ai_company_evaluatuion\\chromedriver_win32\\chromedriver.exe"

In [8]:
from selenium import webdriver

In [34]:
driver = webdriver.Chrome(web_driver_path)
driver.implicitly_wait(3)

In [35]:
driver.get(url)

In [36]:
xpath_industry = {}
xpath_industry["industry"] = """//*[@id="header"]/div[3]/div[2]/div[1]/ul[1]/li[1]/a""" 
xpath_industry["company"]  = {}
xpath_industry["company"][1] = """//*[@id="list_left_aside_id"]/ul/li[2]/a"""
xpath_industry["company"][2] = """//*[@id="contents"]/div/div[2]/div[2]/ul/li[3]/a"""
 
driver.find_element_by_xpath(xpath_industry["industry"]).click()


In [38]:
html = driver.page_source
html

'<html lang="ko"><head>\n    <meta charset="UTF-8">\n    <title>산업 섹션 - Chosunbiz</title>\n    <meta name="description" content="조선비즈는 국내 최고의 온라인 경제 전문 미디어로, 정책, 금융, 부동산, 기업, 글로벌 경제 등 정보를 제공합니다.">\n    <meta name="keywords" content="조선비즈, chosunbiz, 종합미디어">\n    <meta name="author" content="ChosunBiz">\n    <meta name="HITLOG" content="60;7;sec_7">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n\n    <link href="//biz.chosun.com/site/data/rss/rss.xml" rel="alternate" type="application/rss+xml" title="Chosun.com [RSS]">\n    <link href="//news.chosun.com/favicon.ico" rel="icon" type="image/x-icon">\n    <link rel="canonical" href="//biz.chosun.com">\n    <link rel="stylesheet" type="text/css" href="//biz.chosun.com/dhtm/2018/css/header_footer.css">\n    <link rel="stylesheet" type="text/css" href="//biz.chosun.com/dhtm/2018/css/section_default.css">\n\n\t<script type="text/javascript" async="" src="https://www.google-analytics.com/plugins/ga/inpage_linkid.js" id="u

In [9]:
def preprocess_word(x) :
    m = x.replace('%','per')
    m = re.sub(r'(?:\b[0-9a-zA-Zㄱ-ㅎㅏ-ㅣ]\b|[?!\W]+)\s*', ' ', m).strip() 
    m = re.sub('per','%',m)
    m = " ".join(m.split())
    return m

In [10]:
def get_string_from_chosun(bsObject):
    text_list = []
    for i,meta in enumerate(bsObject.find_all("a")):
        text_list.append(meta.get_text())
    return text_list
#     print(i,bsObject.a['on_click'])

In [11]:
def add_company_info_from(s_list,company_dict,verbose=True):
    for s in s_list:
        
        processed_words = []
        for word in s.split(' '):
            new_word = preprocess_word(word)
            if new_word in ["속보","전체보기"]:
                continue
            elif new_word and new_word!='':
                processed_words.append(new_word)
        if processed_words and verbose:print('*'*20);print(processed_words)
        for word in processed_words:
            if word in company_list:
                company_dict[word].append(" ".join(processed_words))
        

In [12]:
company_dict.clear()
chosun_news = get_string_from_chosun(bsObject) 
add_company_info_from(chosun_news,company_dict)


********************
['확', '바뀌는', '롯데쇼핑', '점포', '200개', '폐점하고', '체질개선', '본격', '돌입']
********************
['우한폐렴에', 'MWC', '전격', '취소', '화웨이', '200억원', '날렸다']
********************
['MWC', '바이러스에', '주저앉다', '33년', '사상', '첫', '행사', '취소']
********************
['文대통령', '만난', '이재용', '어깨', '무거워', '고용', '직접', '챙기겠다']
********************
['조원태', '물러나라', '이사회', '장악', '시동거는', '조현아', '연합군']
********************
['폭등하는', '수원 용인', '집값', '1주일', '새', '2%', '올라']
********************
['갤럭시Z', '플립의', '는', '무엇을', '의미할까']
********************
['삼성전자', '갤럭시', '플립', '톰브라운', '에디션', '21일부터', '한정판매']
********************
['BTS', '이어', '기생충', '투자', '성공', '산업은행', '2연타석', '대박']
********************
['우한폐렴이', '무슨', '상관', 'LG생건', '성과급', '지연에', '부글부글']
********************
['실손', '누수', '막으려다', '의료계에', '판판이', '깨진', '보험업계']
********************
['일본서', '첫', '우한폐렴', '사망자', '발생', '80대', '여성']
********************
['알립니다', '사랑받는', '브랜드', '대상', '조선비즈가', '선정합니다']
********************
['제네시스', 'G80', '출시', '연기', '신차', '골든사이클

In [13]:
pprint.pprint(company_dict)
# print(list(company_list))

defaultdict(<class 'list'>,
            {'CJ': ['이재현 CJ 회장 대통령 재계 간담회 첫 참석',
                    'CJ ENM 작년 영업익 2694억원 전 사업 부문 흑자',
                    'CJ ENM 작년 영업익 2694억원 전 사업 부문 흑자'],
             'CJ제일제당': ['CJ제일제당 냉동 간편식 매출 6620억원 점유율 34%로 1위'],
             'KB금융': ['KB금융 위해 5500억 자금 조달'],
             'LG하우시스': ['LG하우시스 호텔 인테리어 시장 공략 나서'],
             'LG화학': ['LG화학 9000억원 회사채 발행 석유화학 시설 자금으로 쓸 듯'],
             'SK이노베이션': ['헝가리 배터리 공장 찾은 SK이노베이션 대표 유럽시장 공략'],
             '금호산업': ['금호산업 작년 영업이익 556억 전년 比 31 7%'],
             '넷마블': ['넷마블 4분기 영업이익 502억원 글로벌 성과에 전년比 32 1% 늘어'],
             '대상': ['알립니다 사랑받는 브랜드 대상 조선비즈가 선정합니다', '대상 마곡에 D센터 건설'],
             '대웅제약': ['대웅제약 개별 매출기준 첫 1조원 돌파 1조 클럽 가입',
                      '대웅제약 개별 매출기준 첫 1조원 돌파 1조 클럽 가입'],
             '두산인프라코어': ['두산인프라코어 지난해 매출 8조원 넘었다 엔진 판매 호조 덕'],
             '롯데쇼핑': ['확 바뀌는 롯데쇼핑 점포 200개 폐점하고 체질개선 본격 돌입'],
             '바른손': ['나흘간 184% 오른 기생충株 바른손 투자경고종목 지정'],
             '삼성전자': ['삼성전자 갤럭시 플립 톰브라운 에디션

In [14]:
def save_json_newsdata(dict_data:dict,url:str,save_dir='crawling_data'):
    save_time = time.strftime('%Y-%m-%d', time.localtime(time.time()))
    crawl_name = url.split("//")[-1].strip('/')
    file_name = f"{save_time}_{crawl_name}.json"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir,file_name)
    with open(save_path,"w") as json_file:
        json.dump(dict_data,json_file)

In [15]:

save_json_newsdata(company_dict,url=url)